# Algorithm to Follow for Performing Visual Odometry

Feature Matching and Outlier rejection using RANSAC

Estimating Fundamental Matrix

Estimating Essential Matrix from Fundamental Matrix

Estimate Camera Pose from Essential Matrix

Check for Cheirality Condition using Triangulation

Perspective-n-Point

Bundle Adjustment

In [88]:
import numpy as np
import scipy
import cv2
import glob
import matplotlib.pyplot as plt
from ReadCameraModel import *
from UndistortImage import *
%matplotlib inline

In [165]:
# Data Preparation

camera_model = glob.glob('C:\\Users\\shant\\dataset\\model\\')
data_files = glob.glob('C:\\Users\\shant\\dataset\\stereo\\centre\\')

def intrinsic_matrix_undistorted_image(images,model_path):
    
    """
    Inputs:
    
    model_path: The path to the model files
    images: The input image
    
    Outputs:
    k_matrix: The intrinsic parameters
    undistorted_image: The undistorted image
    """
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BAYER_GR2BGR)
    
    fx, fy, cx, cy, G_camera_image, LUT = ReadCameraModel(model_path)
    k_matrix = np.zeros((3,3))
    k_matrix[0,0] = fx
    k_matrix[0,2] = cx
    k_matrix[1,1] = fy
    k_matrix[1,2] = cy
    k_matrix[2,2] = 1
    
    undistorted_image = UndistortImage(image, LUT)
    
    return (k_matrix, undistorted_image)

# Estimating the Fundamental and Essential Matrix

def FundamentalMatrix(input_points, output_correspondence, scaling):
    
    """
    Inputs:
    input_points: This is a Nx2 Matrix of (x,y) points
    output_correspondance: This is a Nx2 Matrix of (x',y') points
    scaling: The maximum of the input images width and height
    
    """
    # Normalize the input coordinates with the scaling factor
    
    pts1 = input_points / scaling
    pts2 = output_correspondence / scaling
    
    # List of Fundamental Matrix
    F_list = []
    
    # Transformation matrix for unnormalizing the fundamental matrix
    T = np.array([[1/scaling,0,0],[0,1/scaling,0],[0,0,1]])
    
    # Construct the A matrix 
    first_row_A = np.array([[pts1[:,0][0]*pts2[:,0][0], pts1[:,0][0]*pts2[:,1][0], pts1[:,0][0], pts1[:,1][0]*pts2[:,0][0], pts1[:,1][0]*pts2[:,1][0], pts1[:,1][0], pts2[:,0][0], pts2[:,1][0], 1]])
    second_row_A = np.array([[pts1[:,0][1]*pts2[:,0][1], pts1[:,0][1]*pts2[:,1][1], pts1[:,0][1], pts1[:,1][1]*pts2[:,0][1], pts1[:,1][1]*pts2[:,1][1], pts1[:,1][1], pts2[:,0][1], pts2[:,1][1], 1]])
    third_row_A = np.array([[pts1[:,0][2]*pts2[:,0][2], pts1[:,0][2]*pts2[:,1][2], pts1[:,0][2], pts1[:,1][2]*pts2[:,0][2], pts1[:,1][2]*pts2[:,1][2], pts1[:,1][2], pts2[:,0][2], pts2[:,1][2], 1]]) 
    fourth_row_A = np.array([[pts1[:,0][3]*pts2[:,0][3], pts1[:,0][3]*pts2[:,1][3], pts1[:,0][3], pts1[:,1][3]*pts2[:,0][3], pts1[:,1][3]*pts2[:,1][3], pts1[:,1][3], pts2[:,0][3], pts2[:,1][3], 1]])
    fifth_row_A = np.array([[pts1[:,0][4]*pts2[:,0][4], pts1[:,0][4]*pts2[:,1][4], pts1[:,0][4], pts1[:,1][4]*pts2[:,0][4], pts1[:,1][4]*pts2[:,1][4], pts1[:,1][4], pts2[:,0][4], pts2[:,1][4], 1]])
    sixth_row_A = np.array([[pts1[:,0][5]*pts2[:,0][5], pts1[:,0][5]*pts2[:,1][5], pts1[:,0][5], pts1[:,1][5]*pts2[:,0][5], pts1[:,1][5]*pts2[:,1][5], pts1[:,1][5], pts2[:,0][5], pts2[:,1][5], 1]])
    seventh_row_A = np.array([[pts1[:,0][6]*pts2[:,0][6], pts1[:,0][6]*pts2[:,1][6], pts1[:,0][6], pts1[:,1][6]*pts2[:,0][6], pts1[:,1][6]*pts2[:,1][6], pts1[:,1][6], pts2[:,0][6], pts2[:,1][6], 1]]) 
    eighth_row_A = np.array([[pts1[:,0][7]*pts2[:,0][7], pts1[:,0][7]*pts2[:,1][7], pts1[:,0][7], pts1[:,1][7]*pts2[:,0][7], pts1[:,1][7]*pts2[:,1][7], pts1[:,1][7], pts2[:,0][7], pts2[:,1][7], 1]])
    
    # Stack the rows to create the A matrix
    A = np.vstack((first_row_A,second_row_A,third_row_A,fourth_row_A,fifth_row_A,sixth_row_A,seventh_row_A,eighth_row_A,np.ones(9)))
    # Singular Value Decomposition
    U, S, Vh = np.linalg.svd(A)
    V = Vh.T
    
    # Constructing the fundamental matrix by taking the last column of the V matrix as it corresponds to the nullspace eigenvector
    fundamental_matrix = V[:,-1]
    fundamental_matrix = fundamental_matrix.reshape(3,3)
    
    # Enforcing Rank 2 constraint
    U, sigma, Vh = np.linalg.svd(fundamental_matrix)
    sigma[2] = 0
    fundamental_matrix = np.matmul(U, np.matmul(np.diag(sigma), Vh))
    
    # Unnormalize the fundmental matrix
    fundamental_matrix = np.matmul(np.matmul(T.T, fundamental_matrix), T)
    
    F_list.append(fundamental_matrix)
    return F_list
    
def get_keypoints(image1, image2):
    
    """
    Inputs:
    image1: The left image
    image2: The right image
    
    Outputs:
    left_correspondence:
    right_correspondence:
    
    """
def FundamentalMatrixRansac(input_points, output_correspondence, scaling):
    
    """
    Inputs:
    input_points: This is a randomly sampled input point correspondence
    output_correspondance: This is the output point correspondence
    
    Outputs:
    fundamental_matrix: The refined fundamental matrix after performing RANSAC
    """
    # Convert the correspondences into homogenous coordinates
    pts1 = np.hstack(input_points, np.ones(input_points.shape[0],1))
    pts2 = np.hstack(output_correspondence, np.ones(output_correspondence.shape[0], 1))
    
    # Number of iterations
    iterations = 1000
    
    # The threshold error
    epsilon = 0.01
    
    # The indices with value less than the error
    best_indices = None
    
    # Best Fundamental Matrix
    best_fundamental_matrix = None
    
    # Best Inliers
    best_inliers = 0
    
    for i in range(iterations):
        
        rand_index = np.random.choice(input_points.shape[0], 8, False)
        F = FundamentalMatrix(input_points[rand_index], output_correspondence[rand_index],scaling)
        
        for fundamental_matrix in F:
            
            # Print a List of indices
            indices = np.where(np.abs(np.matmul(pts2, np.matmul(fundamental_matrix, pts.T))).diagonal() < epsilon)[0]
            
            if len(indices) > best_inliers:
                
                best_fundamental_matrix = fundamental_matrix
                best_indices = indices
                best_inliers = len(indices)
    
def EssentialMatrix(fundamental_matrix, calibration_matrix):
    
    """
    Inputs:
    fundamental_matrix: The fundamental matrix that gives the epipolar line on which an input point must lie
    calibration_matrix: The K matrix responsible for projecting an object in camera coordinates to the image coordinate system
    
    Ouput:
    essential_matrix: The essential matrix giving the relation between the 2 image points
    """
    essential_matrix = np.matmul(calibration_matrix.T, np.matmul(fundamental_matrix,calibration_matrix))
    U, sigma, Vh = np.linalg.svd(essential_matrix)
    
    sigma[0] = 1
    sigma[1] = 1
    sigma[2] = 0
    
    essential_matrix = np.matmul(U, np.matmul(np.diag(sigma), Vh))
    
    return essential_matrix

def ExtractCameraPose(essential_matrix):
    
    """
    Inputs:
    essential_matrix: The Essential Matrix
    
    Outputs:
    Camera Poses
    """
    
    W = np.array([[0,-1,0],[1,0,0],[0,0,1]])
    
    # Perform the Singular Value Decomposition of the Essential Matrix
    U, sigma, Vh = np.linalg.svd(essential_matrix)
    
    # Defining the 4 Camera Poses (c1,r1), (c2,r2), (c3,r3), (c4,r4)
    c1 = U[:,3]
    c2 = -U[:,3]
    c3 = U[:,3]
    c4 = -U[:,3]
    r1 = np.dot(U, np.dot(W, Vh))
    r2 = np.dot(U, np.dot(W, Vh))
    r3 = np.dot(U, np.dot(np.transpose(W), Vh))
    r4 = np.dot(U, np.dot(np.transpose(W), Vh))
    
    if np.linalg.det(r1) < 0:
        c1 = -c1
        r1 = -r1
    if np.linalg.det(r2) < 0:
        c2 = -c2
        r2 = -r2
    if np.linalg.det(r3) < 0:
        c3 = -c3
        r3 = -r3
    if np.linalg.det(r4) < 0:
        c4 = -c4
        r4 = -r4
    
    # Reshape the translational matrices from 1x3 --> 3x1
    c1 = c1.reshape(-1,1)
    c2 = c2.reshape(-1,1)
    c3 = c3.reshape(-1,1)
    c4 = c4.reshape(-1,1)
    
    return [[np.array(c1), np.array(c2), np.array(c3), np.array(c4)],[np.array(r1), np.array(r2), np.array(r3), np.array(r4)]]

# Triangulation Check for Cheirality 

def LinearTriangulation(camera_pose1, camera_pose2, pts1, pts2):
    
    """
    Inputs:
    camera_pose1: The camera pose for the first camera (3x4)
    camera_pose2: The camera pose for the second camera (3x4)
    pts1: The points in left image whose correspondance needs to be found in the right image
    pts2: The corresponding points in the right image
    
    Outputs:
    P: x, y, z world coordinates
    reprojection_error: The reprojection error from the world frame to the image frame
    """
    P = []
    reprojection_error = 0
    
    # Constructing the A matrix
    for i in range(pts1.shape[0]):
        
        A =  [pts1[i, 0] * camera_pose1[2, :] - camera_pose1[0, :],
              pts1[i, 1] * camera_pose1[2, :] - camera_pose1[1, :],
              pts2[i, 0] * camera_pose2[2, :] - camera_pose2[0, :],
              pts2[i, 1] * camera_pose2[2, :] - camera_pose2[1, :]]
        
        
        # Perform the singular value decomposition of the A matrix
        U, sigma, Vh = np.linalg.svd(A)
        
        # Extract the last row corresponding the nullspace of the linear equation
        p = Vh[-1,:]
        
        # Convert to homogenous coordinates
        w = p/p[3]
        
        # X,Y,Z co-ordinates in the world frame
        P.append(w[:3])
        
        # Checking the Reprojection Error
        projection1 = np.dot(camera_pose1, w)
        projection2 = np.dot(camera_pose2, w)
        
        # Calculating the Reprojection error
        reprojection_error += np.linalg.norm(projection1[:2]/projection1[-1] - pts1[i])**2 + np.linalg.norm(projection2[:2]/projection[-1] - pts2[i])**2
        
    return np.asarray(P), reprojection_error


In [195]:
A = np.array([[1,2],[4,5],[8,9],[12,13]])
A[2]

array([8, 9])

In [194]:
x = np.array([[1],[2],[3],[4]])
x[-1]

array([4])

In [135]:
A[[2,1,0]]

array([[7, 8],
       [4, 5],
       [1, 2]])

In [173]:
import scipy
A = np.array([[1,8,1,2],[2,9,1,2],[3,10,1,56]])

In [164]:
A = np.array([[1,2,3]])
A.reshape(-1,1).shape

(3, 1)

In [174]:
A[2,:]

array([ 3, 10,  1, 56])